In [1]:
import tensorflow as tf
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import os
import re
import shutil

### file 불러오기

In [2]:
image_dir = os.getcwd() + '\\images'
bbox_dir = os.getcwd() + '\\annotations\\xmls'
seg_dir = os.getcwd() + '\\annotations\\trimaps'

In [3]:
image_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] == '.jpg']

len(image_files)

7378

In [4]:
## channel이 3이 아닌 image는 삭제
for file in image_files:
    image_path = os.path.join(image_dir, file)
    image = Image.open(image_path)
    
    if image.mode != 'RGB':
        print(file, image.mode)
        image = np.asarray(image)
        print(image.shape)
        os.remove(image_path)

In [5]:
image_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] == '.jpg']

len(image_files)

7378

In [6]:
# class list 만들기
class_list = set()

for file in image_files:
    file_name = os.path.splitext(file)[0]
    class_name = re.sub('_\d+', '', file_name)
    class_list.add(class_name)
    
class_list = list(class_list)
class_list.sort()
print(len(class_list))
class_list

37


['Abyssinian',
 'Bengal',
 'Birman',
 'Bombay',
 'British_Shorthair',
 'Egyptian_Mau',
 'Maine_Coon',
 'Persian',
 'Ragdoll',
 'Russian_Blue',
 'Siamese',
 'Sphynx',
 'american_bulldog',
 'american_pit_bull_terrier',
 'basset_hound',
 'beagle',
 'boxer',
 'chihuahua',
 'english_cocker_spaniel',
 'english_setter',
 'german_shorthaired',
 'great_pyrenees',
 'havanese',
 'japanese_chin',
 'keeshond',
 'leonberger',
 'miniature_pinscher',
 'newfoundland',
 'pomeranian',
 'pug',
 'saint_bernard',
 'samoyed',
 'scottish_terrier',
 'shiba_inu',
 'staffordshire_bull_terrier',
 'wheaten_terrier',
 'yorkshire_terrier']

In [7]:
# class labeling
class_idx = {cls:idx for idx, cls in enumerate(class_list)}
class_idx

{'Abyssinian': 0,
 'Bengal': 1,
 'Birman': 2,
 'Bombay': 3,
 'British_Shorthair': 4,
 'Egyptian_Mau': 5,
 'Maine_Coon': 6,
 'Persian': 7,
 'Ragdoll': 8,
 'Russian_Blue': 9,
 'Siamese': 10,
 'Sphynx': 11,
 'american_bulldog': 12,
 'american_pit_bull_terrier': 13,
 'basset_hound': 14,
 'beagle': 15,
 'boxer': 16,
 'chihuahua': 17,
 'english_cocker_spaniel': 18,
 'english_setter': 19,
 'german_shorthaired': 20,
 'great_pyrenees': 21,
 'havanese': 22,
 'japanese_chin': 23,
 'keeshond': 24,
 'leonberger': 25,
 'miniature_pinscher': 26,
 'newfoundland': 27,
 'pomeranian': 28,
 'pug': 29,
 'saint_bernard': 30,
 'samoyed': 31,
 'scottish_terrier': 32,
 'shiba_inu': 33,
 'staffordshire_bull_terrier': 34,
 'wheaten_terrier': 35,
 'yorkshire_terrier': 36}

In [8]:
# train / validation dataset
train_dir = os.path.join(os.getcwd(), 'train')
val_dir = os.path.join(os.getcwd(), 'validation')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [9]:
cnt = 0
previous_class = ''

for file in tqdm(image_files):
    file_name = os.path.splitext(file)[0]
    class_name = re.sub('_\d+', '', file_name)
    if class_name == previous_class:
        cnt += 1
    else:
        cnt = 1
        
    if cnt <= 160:
        cpath = train_dir
    else:
        cpath = val_dir
        
    image_path = os.path.join(image_dir, file)
    shutil.copy(image_path, cpath)
    previous_class = class_name

100%|██████████| 7378/7378 [00:04<00:00, 1534.60it/s]


In [10]:
train_images = os.listdir(train_dir)
val_images = os.listdir(val_dir)

len(train_images), len(val_images)

(5920, 1458)

### TFRecord file 만들기

In [11]:
img_size = 224

In [12]:
# TFRecord 저장할 directory와 file 경로
tfr_dir = os.path.join(os.getcwd(), 'tfrecord')
os.makedirs(tfr_dir, exist_ok=True)

tfr_train_dir = os.path.join(tfr_dir, 'cls_train.tfr')
tfr_val_dir = os.path.join(tfr_dir, 'cls_val.tfr')

# TFRecord writer
writer_train = tf.io.TFRecordWriter(tfr_train_dir)
writer_val = tf.io.TFRecordWriter(tfr_val_dir)

#### TFRecord를 사용하기 위해 필요한 Feature 정의 [[link]](https://www.tensorflow.org/tutorials/load_data/tfrecord?hl=ko)

In [13]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [14]:
## Training data
n_train = 0
train_files = os.listdir(train_dir)

for file in tqdm(train_files):
    train_path = os.path.join(train_dir, file)
    image = Image.open(train_path)
    image = image.resize((img_size, img_size))   # 224, 224
    b_image = image.tobytes()
    
    file_name = os.path.splitext(file)[0]
    class_name = re.sub('_\d+', '', file_name)
    class_num = class_idx[class_name]
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytes_feature(b_image), 
        'cls_num': _int64_feature(class_num)
    }))
   
    writer_train.write(example.SerializeToString())
    n_train += 1
    
writer_train.close()
print(n_train)

100%|██████████| 5920/5920 [00:49<00:00, 120.44it/s]

5920


In [15]:
## Validation data
n_val = 0
val_files = os.listdir(val_dir)

for file in tqdm(val_files):
    val_path = os.path.join(val_dir, file)
    image = Image.open(val_path)
    image = image.resize((img_size, img_size))   # 224, 224
    b_image = image.tobytes()
    
    file_name = os.path.splitext(file)[0]
    class_name = re.sub('_\d+', '', file_name)
    class_num = class_idx[class_name]
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytes_feature(b_image), 
        'cls_num': _int64_feature(class_num)
    }))
   
    writer_val.write(example.SerializeToString())
    n_val += 1
    
writer_val.close()
print(n_val)

100%|██████████| 1458/1458 [00:12<00:00, 115.89it/s]

1458


## Image Classification

In [16]:
# Hyperparameters
n_class = len(class_list)
epochs = 50
batch_size = 40
learning_rate = 0.001
dropout_rate = 0.4
steps_per_epoch = n_train // batch_size
validation_steps = n_val // batch_size + 1

In [17]:
## TFRecord file을 data로 parsing해주는 함수
def parsing_function(tfrecord_serialized):
    features = {'image': tf.io.FixedLenFeature([], tf.string), 
               'cls_num': tf.io.FixedLenFeature([], tf.int64)}
    
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)
    
    image = tf.io.decode_raw(parsed_features['image'], tf.int8)
    image = tf.reshape(image, [img_size, img_size, 3])
    image = tf.cast(image, tf.float32) / 255.
    
    label = tf.cast(parsed_features['cls_num'], tf.int64)
    
    return image, label

In [18]:
# Train Dataset
train_dataset = tf.data.TFRecordDataset(tfr_train_dir)
train_dataset = train_dataset.map(parsing_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(n_train).prefetch(tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size).repeat()

In [19]:
# Validation Dataset
val_dataset = tf.data.TFRecordDataset(tfr_val_dir)
val_dataset = val_dataset.map(parsing_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).repeat()

### Model 구성

In [20]:
def MyModel():
    model = tf.keras.Sequential()
    
    # feature extraction
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_size, img_size, 3)))
    model.add(layers.MaxPool2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPool2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPool2D((2, 2)))
    model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPool2D((2, 2)))
    
    # fully connected
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(n_class, activation='softmax'))
    
    return model

In [21]:
model = MyModel()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), 
             loss='sparse_categorical_crossentropy', 
             metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       2

In [22]:
hist = model.fit(train_dataset, 
                epochs=epochs, 
                steps_per_epoch=steps_per_epoch,
                validation_data=val_dataset, 
                validation_steps=validation_steps)

Epoch 1/50
148/148 [==============================] - 15s 65ms/step - loss: 3.6239 - accuracy: 0.0353 - val_loss: 3.5274 - val_accuracy: 0.0542
Epoch 2/50
148/148 [==============================] - 10s 62ms/step - loss: 3.4471 - accuracy: 0.0696 - val_loss: 3.4063 - val_accuracy: 0.0796
Epoch 3/50
148/148 [==============================] - 10s 62ms/step - loss: 3.2415 - accuracy: 0.1182 - val_loss: 3.3649 - val_accuracy: 0.0947
Epoch 4/50
148/148 [==============================] - 10s 62ms/step - loss: 2.6993 - accuracy: 0.2562 - val_loss: 3.4269 - val_accuracy: 0.1111
Epoch 5/50
148/148 [==============================] - 10s 63ms/step - loss: 1.6979 - accuracy: 0.5245 - val_loss: 4.1185 - val_accuracy: 0.0988
Epoch 6/50
148/148 [==============================] - 10s 63ms/step - loss: 0.8925 - accuracy: 0.7437 - val_loss: 5.8475 - val_accuracy: 0.0926
Epoch 7/50
148/148 [==============================] - 10s 63ms/step - loss: 0.5262 - accuracy: 0.8471 - val_loss: 7.0289 - val_accuracy:

#### -> train dataset의 정확도는 크게 상승하지만 validation dataset의 정확도는 개선이 되지 않음 (Overfitting)

In [23]:
## Batch Normalization 추가
def BN_Model():
    model = tf.keras.Sequential()
    
    # Feature Extraction
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=(img_size, img_size, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPool2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPool2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPool2D((2, 2)))
    
    model.add(layers.Conv2D(256, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPool2D((2, 2)))
    
    model.add(layers.Conv2D(256, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPool2D((2, 2)))
    
    # Fully Connected
    model.add(layers.Flatten())
    model.add(layers.Dense(1024))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Dense(n_class))
    model.add(layers.Softmax())
    
    return model

In [24]:
model = BN_Model()

## Learning Rate Scheduling
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate, 
                                                            decay_steps=steps_per_epoch * 5, 
                                                            decay_rate=0.5, 
                                                            staircase=True)  # 계단식

model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
             loss='sparse_categorical_crossentropy', 
             metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 32)      128       
_________________________________________________________________
re_lu (ReLU)                 (None, 224, 224, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 112, 112, 64)     

In [25]:
hist_1 = model.fit(train_dataset, 
                  epochs=epochs, 
                  steps_per_epoch=steps_per_epoch,
                  validation_data=val_dataset,
                  validation_steps=validation_steps)

Epoch 1/50
148/148 [==============================] - 16s 87ms/step - loss: 3.4758 - accuracy: 0.1128 - val_loss: 7.0787 - val_accuracy: 0.0480
Epoch 2/50
148/148 [==============================] - 15s 84ms/step - loss: 2.7429 - accuracy: 0.2454 - val_loss: 4.5372 - val_accuracy: 0.0665
Epoch 3/50
148/148 [==============================] - 15s 85ms/step - loss: 2.0443 - accuracy: 0.4169 - val_loss: 4.0263 - val_accuracy: 0.1303
Epoch 4/50
148/148 [==============================] - 15s 85ms/step - loss: 1.1455 - accuracy: 0.6769 - val_loss: 3.4663 - val_accuracy: 0.1872
Epoch 5/50
148/148 [==============================] - 15s 85ms/step - loss: 0.3901 - accuracy: 0.9206 - val_loss: 3.5930 - val_accuracy: 0.1838
Epoch 6/50
148/148 [==============================] - 15s 85ms/step - loss: 0.0679 - accuracy: 0.9963 - val_loss: 3.0960 - val_accuracy: 0.2524
Epoch 7/50
148/148 [==============================] - 15s 85ms/step - loss: 0.0208 - accuracy: 1.0000 - val_loss: 3.0940 - val_accuracy:

#### 위 보다는 조금 개선되었지만 여전히 과적합이 있음

### Pretrained MobileNetV2 사용하여 학습

<img src='https://blog.kakaocdn.net/dn/Dcwve/btqCly1YfxG/iQSSoddgoiGV7BHIce7FCK/img.png' width=400>

In [26]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [27]:
MobileNet_V2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
MobileNet_V2.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
________________

Total params: 2,257,984
Trainable params: 2,223,872
Non-trainable params: 34,112
__________________________________________________________________________________________________


In [30]:
def Mobile_V2():
    model = tf.keras.Sequential()
    model.add(MobileNet_V2)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(n_class, activation='softmax'))
    
    return model

In [31]:
model = Mobile_V2()

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate, 
                                                            decay_steps=steps_per_epoch * 5, 
                                                            decay_rate=0.5, 
                                                            staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
             loss='sparse_categorical_crossentropy', 
             metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 37)                47397     
Total params: 2,305,381
Trainable params: 2,271,269
Non-trainable params: 34,112
_________________________________________________________________


In [32]:
hist_2 = model.fit(train_dataset, 
                  epochs=epochs, 
                  steps_per_epoch=steps_per_epoch, 
                  validation_data=val_dataset, 
                  validation_steps=validation_steps)

Epoch 1/50
148/148 [==============================] - 40s 222ms/step - loss: 1.7165 - accuracy: 0.4890 - val_loss: 11.0351 - val_accuracy: 0.0837
Epoch 2/50
148/148 [==============================] - 36s 218ms/step - loss: 0.8588 - accuracy: 0.7267 - val_loss: 15.9151 - val_accuracy: 0.0556
Epoch 3/50
148/148 [==============================] - 38s 232ms/step - loss: 0.5990 - accuracy: 0.8022 - val_loss: 11.8426 - val_accuracy: 0.0871
Epoch 4/50
148/148 [==============================] - 37s 228ms/step - loss: 0.4210 - accuracy: 0.8598 - val_loss: 15.4090 - val_accuracy: 0.0693
Epoch 5/50
148/148 [==============================] - 36s 219ms/step - loss: 0.3612 - accuracy: 0.8814 - val_loss: 13.9628 - val_accuracy: 0.0844
Epoch 6/50
148/148 [==============================] - 35s 212ms/step - loss: 0.1215 - accuracy: 0.9627 - val_loss: 9.6553 - val_accuracy: 0.1550
Epoch 7/50
148/148 [==============================] - 35s 216ms/step - loss: 0.0294 - accuracy: 0.9943 - val_loss: 7.1689 - v

#### 이 전의 모델과 비교해서 큰 진전이 있음

0.1104  ->  0.2675  ->  0.8107